Most of stuff was generated by gpt

In [ ]:
import mne
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from mne.decoding import CSP
from sklearn.decomposition import PCA

# Import PyRiemann for Riemannian Geometry-based methods
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM
from pyriemann.estimation import Shrinkage

# Custom Transformers
class ReshapeTransformer(BaseEstimator, TransformerMixin):
    """Reshapes 3D EEG data to 2D for classifier input."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        n_samples = X.shape[0]
        return X.reshape(n_samples, -1)

class FilterTransformer(BaseEstimator, TransformerMixin):
    """Applies bandpass filter to EEG data."""
    def __init__(self, l_freq=1., h_freq=40., sfreq=256):
        self.l_freq = l_freq
        self.h_freq = h_freq
        self.sfreq = sfreq
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_filtered = mne.filter.filter_data(X, sfreq=self.sfreq, l_freq=self.l_freq, h_freq=self.h_freq)
        return X_filtered

class CSPTransformer(BaseEstimator, TransformerMixin):
    """Applies Common Spatial Patterns (CSP) to EEG data."""
    def __init__(self, n_components=4, reg=None):
        self.n_components = n_components
        self.reg = reg
        self.csp = CSP(n_components=self.n_components, reg=self.reg, log=True, norm_trace=False)
    def fit(self, X, y):
        self.csp.fit(X, y)
        return self
    def transform(self, X):
        return self.csp.transform(X)

class PSDTransformer(BaseEstimator, TransformerMixin):
    """Extracts Power Spectral Density (PSD) features."""
    def __init__(self, sfreq=256, fmin=0.1, fmax=40):
        self.sfreq = sfreq
        self.fmin = fmin
        self.fmax = fmax
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        psd, freqs = mne.time_frequency.psd_array_multitaper(X, sfreq=self.sfreq, fmin=self.fmin, fmax=self.fmax)
        return psd.reshape(psd.shape[0], -1)

class PCATransformer(BaseEstimator, TransformerMixin):
    """Applies PCA to reduce dimensionality."""
    def __init__(self, n_components=10):
        self.pca = PCA(n_components=n_components)
    def fit(self, X, y=None):
        n_samples = X.shape[0]
        X_reshaped = X.reshape(n_samples, -1)
        self.pca.fit(X_reshaped)
        return self
    def transform(self, X):
        n_samples = X.shape[0]
        X_reshaped = X.reshape(n_samples, -1)
        return self.pca.transform(X_reshaped)

class LogVarianceTransformer(BaseEstimator, TransformerMixin):
    """Computes the log-variance of each channel."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # X is of shape (n_samples, n_channels, n_times)
        log_var = np.log(np.var(X, axis=2))
        return log_var

# Define Pipelines
pipelines = {}

# Existing Pipelines with Necessary Adjustments

# 1. Raw Data + SVM (Reshaping needed)
pipelines['Raw_SVM'] = Pipeline([
    ('reshape', ReshapeTransformer()),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 2. Raw Data + Random Forest (Reshaping needed)
pipelines['Raw_RF'] = Pipeline([
    ('reshape', ReshapeTransformer()),
    ('rf', RandomForestClassifier())
])

# 3. Filtered Data + SVM (Reshaping needed)
pipelines['Filtered_SVM'] = Pipeline([
    ('filter', FilterTransformer(l_freq=1., h_freq=40.)),
    ('reshape', ReshapeTransformer()),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 4. Filtered Data + Random Forest (Reshaping needed)
pipelines['Filtered_RF'] = Pipeline([
    ('filter', FilterTransformer(l_freq=1., h_freq=40.)),
    ('reshape', ReshapeTransformer()),
    ('rf', RandomForestClassifier())
])

# 5. CSP + LDA (No reshaping needed)
pipelines['CSP_LDA'] = Pipeline([
    ('filter', FilterTransformer(l_freq=8., h_freq=30.)),
    ('csp', CSPTransformer(n_components=4)),
    ('lda', LDA())
])

# 6. CSP + SVM (No reshaping needed)
pipelines['CSP_SVM'] = Pipeline([
    ('filter', FilterTransformer(l_freq=8., h_freq=30.)),
    ('csp', CSPTransformer(n_components=4)),
    ('svm', SVC())
])

# 7. PSD Features + KNN (No reshaping needed)
pipelines['PSD_KNN'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 8. PSD Features + Logistic Regression (No reshaping needed)
pipelines['PSD_LogReg'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression())
])

# 9. PCA + SVM (Reshaping is handled inside PCATransformer)
pipelines['PCA_SVM'] = Pipeline([
    ('pca', PCATransformer(n_components=20)),
    ('svm', SVC())
])

# 10. PCA + Random Forest (Reshaping is handled inside PCATransformer)
pipelines['PCA_RF'] = Pipeline([
    ('pca', PCATransformer(n_components=20)),
    ('rf', RandomForestClassifier())
])

# 11. Bandpass Filter + CSP + SVM (No reshaping needed)
pipelines['Bandpass_CSP_SVM'] = Pipeline([
    ('filter', FilterTransformer(l_freq=8., h_freq=12.)),
    ('csp', CSPTransformer(n_components=4)),
    ('svm', SVC())
])

# 12. Bandpass Filter + CSP + Random Forest (No reshaping needed)
pipelines['Bandpass_CSP_RF'] = Pipeline([
    ('filter', FilterTransformer(l_freq=8., h_freq=12.)),
    ('csp', CSPTransformer(n_components=4)),
    ('rf', RandomForestClassifier())
])

# 13. Filtered Data + LDA (Reshaping needed)
pipelines['Filtered_LDA'] = Pipeline([
    ('filter', FilterTransformer(l_freq=1., h_freq=40.)),
    ('reshape', ReshapeTransformer()),
    ('lda', LDA())
])

# 14. Raw Data + Logistic Regression (Reshaping needed)
pipelines['Raw_LogReg'] = Pipeline([
    ('reshape', ReshapeTransformer()),
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression())
])

# 15. PSD Features + SVM (No reshaping needed)
pipelines['PSD_SVM'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 16. PSD Features + Random Forest (No reshaping needed)
pipelines['PSD_RF'] = Pipeline([
    ('psd', PSDTransformer()),
    ('rf', RandomForestClassifier())
])

# 17. CSP + KNN (No reshaping needed)
pipelines['CSP_KNN'] = Pipeline([
    ('filter', FilterTransformer(l_freq=8., h_freq=30.)),
    ('csp', CSPTransformer(n_components=4)),
    ('knn', KNeighborsClassifier())
])

# 18. PCA + LDA (Reshaping is handled inside PCATransformer)
pipelines['PCA_LDA'] = Pipeline([
    ('pca', PCATransformer(n_components=20)),
    ('lda', LDA())
])

pipelines['Filtered_KNN'] = Pipeline([
    ('filter', FilterTransformer(l_freq=1., h_freq=40.)),
    ('reshape', ReshapeTransformer()),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 20. CSP + Logistic Regression (No reshaping needed)
pipelines['CSP_LogReg'] = Pipeline([
    ('filter', FilterTransformer(l_freq=8., h_freq=30.)),
    ('csp', CSPTransformer(n_components=4)),
    ('logreg', LogisticRegression())
])

# New Pipelines from MOABB Research

# 21. ACM + TS + SVM (Adaptive Covariance Matrix + Tangent Space + SVM)
pipelines['ACM_TS_SVM'] = Pipeline([
    ('cov', Covariances(estimator='oas')),  # Adaptive covariance estimation
    ('ts', TangentSpace()),
    ('svm', SVC())
])

# 22. DLCSPauto + shLDA (CSP with automatic regularization + Shrinkage LDA)
pipelines['DLCSPauto_shLDA'] = Pipeline([
    ('filter', FilterTransformer(l_freq=8., h_freq=30.)),
    ('csp', CSPTransformer(n_components=4, reg='ledoit_wolf')),
    ('lda', LDA(solver='lsqr', shrinkage='auto'))
])

# 23. FilterBank + SVM (Filter bank CSP + SVM)
# Note: Implementing Filter Bank CSP requires custom implementation
# Here, we simulate it with multiple filters and concatenated CSP features
class FilterBankCSPTransformer(BaseEstimator, TransformerMixin):
# 19. Filtered Data + KNN (Reshaping needed)
    """Applies CSP over multiple frequency bands and concatenates features."""
    def __init__(self, frequency_bands, n_components=4):
        self.frequency_bands = frequency_bands
        self.n_components = n_components
        self.csp_list = []
    def fit(self, X, y):
        self.csp_list = []
        for band in self.frequency_bands:
            csp = CSP(n_components=self.n_components, reg=None, log=True, norm_trace=False)
            X_filtered = mne.filter.filter_data(X, sfreq=256, l_freq=band[0], h_freq=band[1])
            csp.fit(X_filtered, y)
            self.csp_list.append((band, csp))
        return self
    def transform(self, X):
        features = []
        for band, csp in self.csp_list:
            X_filtered = mne.filter.filter_data(X, sfreq=256, l_freq=band[0], h_freq=band[1])
            features.append(csp.transform(X_filtered))
        return np.concatenate(features, axis=1)

frequency_bands = [(4, 8), (8, 12), (12, 16), (16, 20), (20, 24), (24, 28), (28, 32)]
pipelines['FilterBank_SVM'] = Pipeline([
    ('fbcsp', FilterBankCSPTransformer(frequency_bands=frequency_bands, n_components=2)),
    ('svm', SVC())
])

# 24. FgMDM (Filter bank geometric MDM)
# Implementing FgMDM requires complex geometric computations
# Here we simulate it with multiple bands and MDM
class FilterBankMDM(BaseEstimator, TransformerMixin):
    """Applies MDM over multiple frequency bands and averages distances."""
    def __init__(self, frequency_bands):
        self.frequency_bands = frequency_bands
        self.mdms = []
    def fit(self, X, y):
        self.mdms = []
        for band in self.frequency_bands:
            mdm = MDM()
            X_filtered = mne.filter.filter_data(X, sfreq=256, l_freq=band[0], h_freq=band[1])
            cov = Covariances().transform(X_filtered)
            mdm.fit(cov, y)
            self.mdms.append((band, mdm))
        return self
    def transform(self, X):
        distances = []
        for band, mdm in self.mdms:
            X_filtered = mne.filter.filter_data(X, sfreq=256, l_freq=band[0], h_freq=band[1])
            cov = Covariances().transform(X_filtered)
            dist = mdm._predict_distances(cov)
            distances.append(dist)
        # Average distances over bands
        avg_distances = np.mean(distances, axis=0)
        return avg_distances

pipelines['FgMDM'] = Pipeline([
    ('fbgmdm', FilterBankMDM(frequency_bands=frequency_bands))
    # Note: MDM classifier is integrated within the transformer
])

# 25. LogVariance + LDA (No reshaping needed)
pipelines['LogVar_LDA'] = Pipeline([
    ('logvar', LogVarianceTransformer()),
    ('lda', LDA())
])

# 26. LogVariance + SVM (No reshaping needed)
pipelines['LogVar_SVM'] = Pipeline([
    ('logvar', LogVarianceTransformer()),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 27. MDM (Covariance estimation + MDM classifier)
pipelines['MDM'] = Pipeline([
    ('cov', Covariances()),
    ('mdm', MDM())
])
# 28. TRCSP + LDA (Temporally Regularized CSP + LDA)
# Implementing TRCSP is complex; we can simulate with regular CSP
pipelines['TRCSP_LDA'] = Pipeline([
    ('filter', FilterTransformer(l_freq=8., h_freq=30.)),
    ('csp', CSPTransformer(n_components=4, reg='oas')),  # Use regularization
    ('lda', LDA())
])

# 29. TS + EL (Tangent Space + Ensemble Learning)
from sklearn.ensemble import VotingClassifier
pipelines['TS_EL'] = Pipeline([
    ('cov', Covariances()),
    ('ts', TangentSpace()),
    ('ensemble', VotingClassifier([
        ('lda', LDA()),
        ('svm', SVC(probability=True)),
        ('rf', RandomForestClassifier())
    ], voting='soft'))
])

# 30. TS + LR (Tangent Space + Logistic Regression)
pipelines['TS_LR'] = Pipeline([
    ('cov', Covariances()),
    ('ts', TangentSpace()),
    ('logreg', LogisticRegression(max_iter=1000))
])

# 31. TS + SVM (Tangent Space + SVM)
pipelines['TS_SVM'] = Pipeline([
    ('cov', Covariances()),
    ('ts', TangentSpace()),
    ('svm', SVC())
])

# Training Function with K-Fold Cross-Validation
def train_and_evaluate(X, y, pipelines, n_splits=5):
    """
    Trains and evaluates multiple pipelines using k-fold cross-validation.

    Parameters:
    - X: EEG data array of shape (n_samples, n_channels, n_times)
    - y: Labels array of shape (n_samples,)
    - pipelines: Dictionary of sklearn Pipelines
    - n_splits: Number of folds for cross-validation

    Returns:
    - results: Dictionary containing cross-validation scores for each pipeline
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    results = {}

    for name, pipeline in pipelines.items():
        print(f"Evaluating pipeline: {name}")
        try:
            scores = cross_val_score(pipeline, X, y, cv=kf, scoring='accuracy', n_jobs=-1)
            print(f"Scores: {scores}")
            print(f"Mean accuracy: {np.mean(scores):.4f}")
            results[name] = scores
        except Exception as e:
            print(f"An error occurred in pipeline {name}: {e}")
            results[name] = None
    return results

# Example Usage
# Assuming `raw` is your MNE Raw object and `labels` is your labels array
# Preprocess raw data to get numpy array
def preprocess_raw(raw, event_id, tmin, tmax):
    """
    Extracts epochs and returns data and labels.

    Parameters:
    - raw: MNE Raw object
    - event_id: Dictionary mapping event names to event IDs
    - tmin: Start time before event
    - tmax: End time after event

    Returns:
    - X: EEG data array of shape (n_epochs, n_channels, n_times)
    - y: Labels array of shape (n_epochs,)
    """
    events = mne.find_events(raw)
    picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False)
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks, baseline=None, preload=True)
    X = epochs.get_data()  # shape: (n_epochs, n_channels, n_times)
    y = epochs.events[:, -1]
    return X, y

# Example call
# X, y = preprocess_raw(raw, event_id={'Stimulus': 1}, tmin=0, tmax=1)
# results = train_and_evaluate(X, y, pipelines, n_splits=5)



In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from mne.decoding import CSP, Vectorizer
from sklearn.decomposition import PCA
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

# Custom Transformers
class ReshapeTransformer(BaseEstimator, TransformerMixin):
    """Reshapes 3D EEG data to 2D for classifier input."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        n_samples = X.shape[0]
        return X.reshape(n_samples, -1)

class HilbertTransformer(BaseEstimator, TransformerMixin):
    """Applies the Hilbert transform to EEG data."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # Apply Hilbert transform along the time axis
        analytic_signal = mne.filter.hilbert(X, picks=None, envelope=False)
        amplitude_envelope = np.abs(analytic_signal)
        return amplitude_envelope

class TimeDomainTransformer(BaseEstimator, TransformerMixin):
    """Computes time-domain features like mean, variance, skewness."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # Compute features along the time axis
        mean = np.mean(X, axis=2)
        var = np.var(X, axis=2)
        skewness = np.mean(((X - mean[:, :, np.newaxis]) ** 3), axis=2) / (var ** 1.5)
        features = np.concatenate((mean, var, skewness), axis=1)
        return features

class MeanAmplitudeTransformer(BaseEstimator, TransformerMixin):
    """Computes the mean amplitude of the signal."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        mean_amplitude = np.mean(np.abs(X), axis=2)
        return mean_amplitude

class ARTransformer(BaseEstimator, TransformerMixin):
    """Computes Autoregressive (AR) coefficients."""
    def __init__(self, order=5):
        self.order = order
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        from statsmodels.tsa.ar_model import AutoReg
        n_samples, n_channels, n_times = X.shape
        ar_features = np.zeros((n_samples, n_channels * self.order))
        for i in range(n_samples):
            for j in range(n_channels):
                model = AutoReg(X[i, j, :], lags=self.order, old_names=False)
                model_fit = model.fit()
                ar_coeffs = model_fit.params[1:]  # Exclude intercept
                ar_features[i, j * self.order:(j + 1) * self.order] = ar_coeffs
        return ar_features

class STFTTransformer(BaseEstimator, TransformerMixin):
    """Computes Short-Time Fourier Transform (STFT)."""
    def __init__(self, n_fft=256):
        self.n_fft = n_fft
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        from scipy.signal import stft
        n_samples, n_channels, n_times = X.shape
        stft_features = []
        for i in range(n_samples):
            sample_features = []
            for j in range(n_channels):
                f, t, Zxx = stft(X[i, j, :], nperseg=self.n_fft)
                sample_features.append(np.abs(Zxx).flatten())
            stft_features.append(np.concatenate(sample_features))
        return np.array(stft_features)

class WaveletTransformer(BaseEstimator, TransformerMixin):
    """Applies Wavelet Transform."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        import pywt
        n_samples, n_channels, n_times = X.shape
        wavelet_features = []
        for i in range(n_samples):
            sample_features = []
            for j in range(n_channels):
                coeffs = pywt.wavedec(X[i, j, :], 'db4', level=3)
                coeffs_flat = np.concatenate([c.flatten() for c in coeffs])
                sample_features.append(coeffs_flat)
            wavelet_features.append(np.concatenate(sample_features))
        return np.array(wavelet_features)

class WPDTransformer(BaseEstimator, TransformerMixin):
    """Applies Wavelet Packet Decomposition (WPD)."""
import mne
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        import pywt
        n_samples, n_channels, n_times = X.shape
        wpd_features = []
            sample_features = []
        for i in range(n_samples):
                wp = pywt.WaveletPacket(X[i, j, :], 'db4', mode='symmetric', maxlevel=3)
            for j in range(n_channels):
                nodes = wp.get_level(3, order='freq')
                coeffs = np.array([n.data for n in nodes], 'd')
            wpd_features.append(np.concatenate(sample_features))
        return np.array(wpd_features)

class FFTTransformer(BaseEstimator, TransformerMixin):
    """Computes Fast Fourier Transform (FFT)."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        fft_coeffs = np.fft.rfft(X, axis=2)
        fft_features = np.abs(fft_coeffs)
        return fft_features.reshape(X.shape[0], -1)

class CARTransformer(BaseEstimator, TransformerMixin):
    """Applies Common Average Referencing (CAR)."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        car = X - np.mean(X, axis=1, keepdims=True)
        return car

class LaplacianTransformer(BaseEstimator, TransformerMixin):
    """Applies Laplacian spatial filtering."""
    def __init__(self, info):
        self.info = info
        self.laplacian = None
    def fit(self, X, y=None):
        from mne.channels import make_standard_montage
        montage = make_standard_montage('standard_1020')
        self.info.set_montage(montage)
        return self
    def transform(self, X):
        laplacian = mne.preprocessing.compute_current_source_density(self.info)
        return laplacian.get_data()

class ICATransformer(BaseEstimator, TransformerMixin):
    """Applies Independent Component Analysis (ICA)."""
    def __init__(self, n_components=15, random_state=42):
        self.n_components = n_components
        self.random_state = random_state
        self.ica = None
    def fit(self, X, y=None):
        n_samples, n_channels, n_times = X.shape
        self.ica = mne.preprocessing.ICA(n_components=self.n_components, random_state=self.random_state)
        X_concat = np.concatenate(X, axis=1)
        info = mne.create_info(n_channels, sfreq=256, ch_types='eeg')
        raw = mne.io.RawArray(X_concat, info)
        self.ica.fit(raw)
        return self
    def transform(self, X):
        n_samples, n_channels, n_times = X.shape
        X_transformed = []
        for i in range(n_samples):
            info = mne.create_info(n_channels, sfreq=256, ch_types='eeg')
            raw = mne.io.RawArray(X[i], info)
            X_ica = self.ica.apply(raw.copy(), exclude=[])
            X_transformed.append(X_ica.get_data())
        return np.array(X_transformed)

# Define Pipelines
pipelines = {}

# Existing pipelines adjusted to remove bandpass filtering
# (assuming bandpass filtering is done separately)

# 1. CSP + Logistic Regression (No reshaping needed)
pipelines['CSP_LogReg'] = Pipeline([
    ('csp', CSP(n_components=4, reg=None, log=True, norm_trace=False)),
    ('logreg', LogisticRegression())
])

# 2. PSD + SVM (No reshaping needed)
pipelines['PSD_SVM'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 3. Time Domain Features + Random Forest (Reshaping needed)
pipelines['TimeDomain_RF'] = Pipeline([
    ('time_features', TimeDomainTransformer()),
    ('rf', RandomForestClassifier())
])

# 4. Hilbert Transform + k-NN (No reshaping needed)
pipelines['Hilbert_KNN'] = Pipeline([
    ('hilbert', HilbertTransformer()),
    ('reshape', ReshapeTransformer()),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 5. Wavelet Transform + PSD + Naive Bayes
pipelines['Wavelet_PSD_NB'] = Pipeline([
    ('wavelet', WaveletTransformer()),
    ('psd', PSDTransformer()),
    ('nb', GaussianNB())
])

# 6. CAR + CSP + Decision Tree
pipelines['CAR_CSP_DT'] = Pipeline([
    ('car', CARTransformer()),
    ('csp', CSP(n_components=4, reg=None, log=True, norm_trace=False)),
    ('dt', DecisionTreeClassifier())
])

# 7. ICA + Time Domain Features + SVM
pipelines['ICA_TimeDomain_SVM'] = Pipeline([
    ('ica', ICATransformer(n_components=15)),
    ('time_features', TimeDomainTransformer()),
    ('svm', SVC())
])

# 8. CSP + LDA (No reshaping needed)
pipelines['CSP_LDA'] = Pipeline([
    ('csp', CSP(n_components=4, reg=None, log=True, norm_trace=False)),
    ('lda', LDA())
])

# 9. PSD + Gradient Boosting
pipelines['PSD_GB'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('gb', GradientBoostingClassifier())
])

# 10. CAR + Riemannian Geometry Features + Logistic Regression
pipelines['CAR_Riemann_LogReg'] = Pipeline([
    ('car', CARTransformer()),
    ('cov', Covariances()),
    ('ts', TangentSpace()),
    ('logreg', LogisticRegression(max_iter=1000))
])

# 11. STFT + SVM
pipelines['STFT_SVM'] = Pipeline([
    ('stft', STFTTransformer(n_fft=256)),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 12. Laplacian + Wavelet Packet Decomposition + Random Forest
# Note: LaplacianTransformer requires EEG info object
# Adjust accordingly in your code
# Assuming `info` is available
pipelines['Laplacian_WPD_RF'] = Pipeline([
    # ('laplacian', LaplacianTransformer(info)),  # Uncomment and set info appropriately
    ('wpd', WPDTransformer()),
    ('rf', RandomForestClassifier())
])

# 13. Hilbert Transform + CNN
# Note: Implementing CNNs requires deep learning frameworks like TensorFlow or PyTorch
# Sklearn does not support CNNs directly, so we'll skip this pipeline or note its complexity

# 14. Time-Frequency Analysis (Morlet Wavelets) + k-NN
class MorletWaveletTransformer(BaseEstimator, TransformerMixin):
    """Applies time-frequency analysis using Morlet wavelets."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        freqs = np.linspace(8, 30, num=22)  # Frequencies from 8 to 30 Hz
        n_samples, n_channels, n_times = X.shape
        power_features = []
        for i in range(n_samples):
            sample_features = []
            for j in range(n_channels):
                power = mne.time_frequency.tfr_array_morlet(X[i:i+1, j:j+1, :], sfreq=256, freqs=freqs, n_cycles=7, output='power')
                sample_features.append(power.flatten())
            power_features.append(np.concatenate(sample_features))
        return np.array(power_features)

pipelines['Morlet_KNN'] = Pipeline([
    ('morlet', MorletWaveletTransformer()),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 15. AR Coefficients + Logistic Regression
pipelines['AR_LogReg'] = Pipeline([
    ('ar', ARTransformer(order=5)),
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000))
])

# 16. RNN implementation is complex and not directly supported in scikit-learn
# We'll skip this pipeline or note its complexity

# 17. Mean Amplitude Features + Naive Bayes
pipelines['MeanAmp_NB'] = Pipeline([
    ('mean_amp', MeanAmplitudeTransformer()),
    ('nb', GaussianNB())
])

# 18. ICA + PSD + SVM
pipelines['ICA_PSD_SVM'] = Pipeline([
    ('ica', ICATransformer(n_components=15)),
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 19. FFT + Random Forest
pipelines['FFT_RF'] = Pipeline([
    ('fft', FFTTransformer()),
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier())
])

# 20. Time-Frequency Features + LDA
# Using STFT for time-frequency features
pipelines['TimeFreq_LDA'] = Pipeline([
    ('stft', STFTTransformer(n_fft=256)),
    ('scaler', StandardScaler()),
    ('lda', LDA())
])

# Training Function with K-Fold Cross-Validation
def train_and_evaluate(X, y, pipelines, n_splits=5):
    """
    Trains and evaluates multiple pipelines using k-fold cross-validation.

    Parameters:
    - X: EEG data array of shape (n_samples, n_channels, n_times)
    - y: Labels array of shape (n_samples,)
    - pipelines: Dictionary of sklearn Pipelines
    - n_splits: Number of folds for cross-validation

    Returns:
    - results: Dictionary containing cross-validation scores for each pipeline
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    results = {}

    for name, pipeline in pipelines.items():
        print(f"Evaluating pipeline: {name}")
        try:
            scores = cross_val_score(pipeline, X, y, cv=kf, scoring='accuracy', n_jobs=-1)
            print(f"Scores: {scores}")
            print(f"Mean accuracy: {np.mean(scores):.4f}")
            results[name] = scores
        except Exception as e:
            print(f"An error occurred in pipeline {name}: {e}")
            results[name] = None
    return results

# Example Usage
# Assuming `X` and `y` are your EEG data and labels
# Adjust the code to match your data loading and preprocessing steps

# Example call
# results = train_and_evaluate(X, y, pipelines, n_splits=5)

                sample_features.append(coeffs.flatten())

su NN ats   

In [ ]:
import mne
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from mne.decoding import CSP
from sklearn.decomposition import PCA
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

# Import for Deep Learning and XGBoost
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, SeparableConv2D, BatchNormalization, Activation, AveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import xgboost as xgb

# Custom Transformers
class ReshapeTransformer(BaseEstimator, TransformerMixin):
    """Reshapes 3D EEG data to 2D for classifier input."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        n_samples = X.shape[0]
        return X.reshape(n_samples, -1)

class HilbertTransformer(BaseEstimator, TransformerMixin):
    """Applies the Hilbert transform to EEG data."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # Apply Hilbert transform along the time axis
        analytic_signal = mne.filter.hilbert(X, picks=None, envelope=False)
        amplitude_envelope = np.abs(analytic_signal)
        return amplitude_envelope

class TimeDomainTransformer(BaseEstimator, TransformerMixin):
    """Computes time-domain features like mean, variance, skewness."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # Compute features along the time axis
        mean = np.mean(X, axis=2)
        var = np.var(X, axis=2)
        skewness = np.mean(((X - mean[:, :, np.newaxis]) ** 3), axis=2) / (var ** 1.5)
        features = np.concatenate((mean, var, skewness), axis=1)
        return features

class MeanAmplitudeTransformer(BaseEstimator, TransformerMixin):
    """Computes the mean amplitude of the signal."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        mean_amplitude = np.mean(np.abs(X), axis=2)
        return mean_amplitude

class ARTransformer(BaseEstimator, TransformerMixin):
    """Computes Autoregressive (AR) coefficients."""
    def __init__(self, order=5):
        self.order = order
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        from statsmodels.tsa.ar_model import AutoReg
        n_samples, n_channels, n_times = X.shape
        ar_features = np.zeros((n_samples, n_channels * self.order))
        for i in range(n_samples):
            for j in range(n_channels):
                model = AutoReg(X[i, j, :], lags=self.order, old_names=False)
                model_fit = model.fit()
                ar_coeffs = model_fit.params[1:]  # Exclude intercept
                ar_features[i, j * self.order:(j + 1) * self.order] = ar_coeffs
        return ar_features

class STFTTransformer(BaseEstimator, TransformerMixin):
    """Computes Short-Time Fourier Transform (STFT)."""
    def __init__(self, n_fft=256):
        self.n_fft = n_fft
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        from scipy.signal import stft
        n_samples, n_channels, n_times = X.shape
        stft_features = []
        for i in range(n_samples):
            sample_features = []
            for j in range(n_channels):
                f, t, Zxx = stft(X[i, j, :], nperseg=self.n_fft)
                sample_features.append(np.abs(Zxx).flatten())
            stft_features.append(np.concatenate(sample_features))
        return np.array(stft_features)

class WaveletTransformer(BaseEstimator, TransformerMixin):
    """Applies Wavelet Transform."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        import pywt
        n_samples, n_channels, n_times = X.shape
        wavelet_features = []
        for i in range(n_samples):
            sample_features = []
            for j in range(n_channels):
                coeffs = pywt.wavedec(X[i, j, :], 'db4', level=3)
                coeffs_flat = np.concatenate([c.flatten() for c in coeffs])
                sample_features.append(coeffs_flat)
            wavelet_features.append(np.concatenate(sample_features))
        return np.array(wavelet_features)

class WPDTransformer(BaseEstimator, TransformerMixin):
    """Applies Wavelet Packet Decomposition (WPD)."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        import pywt
        n_samples, n_channels, n_times = X.shape
        wpd_features = []
        for i in range(n_samples):
            sample_features = []
            for j in range(n_channels):
                wp = pywt.WaveletPacket(X[i, j, :], 'db4', mode='symmetric', maxlevel=3)
                nodes = wp.get_level(3, order='freq')
                coeffs = np.array([n.data for n in nodes], 'd')
                sample_features.append(coeffs.flatten())
            wpd_features.append(np.concatenate(sample_features))
        return np.array(wpd_features)

class FFTTransformer(BaseEstimator, TransformerMixin):
    """Computes Fast Fourier Transform (FFT)."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        fft_coeffs = np.fft.rfft(X, axis=2)
        fft_features = np.abs(fft_coeffs)
        return fft_features.reshape(X.shape[0], -1)

class CARTransformer(BaseEstimator, TransformerMixin):
    """Applies Common Average Referencing (CAR)."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        car = X - np.mean(X, axis=1, keepdims=True)
        return car

class ICATransformer(BaseEstimator, TransformerMixin):
    """Applies Independent Component Analysis (ICA)."""
    def __init__(self, n_components=15, random_state=42):
        self.n_components = n_components
        self.random_state = random_state
        self.ica = None
    def fit(self, X, y=None):
        n_samples, n_channels, n_times = X.shape
        X_concat = X.reshape(n_samples * n_channels, n_times)
        self.ica = mne.preprocessing.ICA(n_components=self.n_components, random_state=self.random_state, max_iter='auto')
        info = mne.create_info(ch_names=['eeg'] * X_concat.shape[0], sfreq=256, ch_types='eeg')
        raw = mne.io.RawArray(X_concat, info)
        self.ica.fit(raw)
        return self
    def transform(self, X):
        n_samples, n_channels, n_times = X.shape
        X_transformed = []
        for i in range(n_samples):
            data = X[i]
            info = mne.create_info(ch_names=['eeg'] * n_channels, sfreq=256, ch_types='eeg')
            raw = mne.io.RawArray(data, info)
            raw_ica = self.ica.apply(raw.copy(), exclude=[])
            X_transformed.append(raw_ica.get_data())
        return np.array(X_transformed)

class EEGNetTransformer(BaseEstimator, TransformerMixin):
    """Transformer that reshapes data for EEGNet."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # Reshape X to (n_samples, n_channels, n_times, 1)
        n_samples, n_channels, n_times = X.shape
        X_reshaped = X.reshape(n_samples, n_channels, n_times, 1)
        return X_reshaped

class MorletWaveletTransformer(BaseEstimator, TransformerMixin):
    """Applies time-frequency analysis using Morlet wavelets."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        freqs = np.linspace(8, 30, num=22)  # Frequencies from 8 to 30 Hz
        n_samples, n_channels, n_times = X.shape
        power_features = []
        for i in range(n_samples):
            sample_features = []
            for j in range(n_channels):
                power = mne.time_frequency.tfr_array_morlet(X[i:i+1, j:j+1, :], sfreq=256, freqs=freqs, n_cycles=7, output='power')
                sample_features.append(power.flatten())
            power_features.append(np.concatenate(sample_features))
        return np.array(power_features)

# Define Pipelines
pipelines = {}

# Existing pipelines adjusted to remove bandpass filtering
# (assuming bandpass filtering is done separately)

# 1. CSP + Logistic Regression (No reshaping needed)
pipelines['CSP_LogReg'] = Pipeline([
    ('csp', CSP(n_components=4, reg=None, log=True, norm_trace=False)),
    ('logreg', LogisticRegression(max_iter=1000))
])

# 2. PSD + SVM (No reshaping needed)
pipelines['PSD_SVM'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# ... [Include all previously defined pipelines here] ...

# 19. EEGNet (Deep CNN)
def create_eegnet_model(n_channels, n_times, n_classes):
    """Creates an EEGNet model."""
    input_shape = (n_channels, n_times, 1)
    inputs = Input(shape=input_shape)

    # Block 1
    x = Conv2D(16, (1, 64), padding='same',
               use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = DepthwiseConv2D((n_channels, 1), use_bias=False,
                        depth_multiplier=2,
                        depthwise_constraint=max_norm(1.))(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)
    x = AveragePooling2D((1, 4))(x)
    x = Dropout(0.25)(x)

    # Block 2
    x = SeparableConv2D(32, (1, 16), use_bias=False, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)
    x = AveragePooling2D((1, 8))(x)
    x = Dropout(0.25)(x)

    x = Flatten()(x)

    x = Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Placeholder function to create the model with the correct input dimensions
def create_eegnet_model_wrapper():
    n_channels = X.shape[1]
    n_times = X.shape[2]
    n_classes = len(np.unique(y))
    return create_eegnet_model(n_channels, n_times, n_classes)

pipelines['EEGNet_CNN'] = Pipeline([
    ('reshape', EEGNetTransformer()),
    ('eegnet', KerasClassifier(build_fn=create_eegnet_model_wrapper, epochs=50, batch_size=16, verbose=0))
])

# 20. PSD Features + XGBoost Classifier
pipelines['PSD_XGB'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
])

# 21. Ensemble Methods (Stacking Classifier)
pipelines['Stacking'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('stacking', StackingClassifier(
        estimators=[
            ('svm', SVC(probability=True)),
            ('rf', RandomForestClassifier()),
            ('knn', KNeighborsClassifier())
        ],
        final_estimator=LogisticRegression(max_iter=1000),
        cv=5
    ))
])

# 22. Sparse Representation Classification (SRC)
# Note: Implementing SRC requires additional libraries or custom code.
# For illustration purposes, we will simulate SRC using Lasso regression.

from sklearn.linear_model import Lasso

class SRCClassifier(BaseEstimator, ClassifierMixin):
    """Simulates a Sparse Representation Classifier using Lasso."""
    def __init__(self, alpha=0.1):
        self.alpha = alpha
        self.classes_ = None
        self.dictionary_ = None
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.dictionary_ = {}
        for cls in self.classes_:
            self.dictionary_[cls] = X[y == cls].T
        return self
    def predict(self, X):
        preds = []
        for x in X:
            residuals = []
            for cls in self.classes_:
                lasso = Lasso(alpha=self.alpha, max_iter=1000)
                lasso.fit(self.dictionary_[cls], x)
                reconstruction = lasso.predict(self.dictionary_[cls])
                residual = np.linalg.norm(x - reconstruction)
                residuals.append(residual)
            preds.append(self.classes_[np.argmin(residuals)])
        return np.array(preds)

pipelines['SRC'] = Pipeline([
    ('reshape', ReshapeTransformer()),
    ('scaler', StandardScaler()),
    ('src', SRCClassifier(alpha=0.1))
])

# 23. Multilayer Perceptron (MLP) Neural Network
pipelines['MLP_NN'] = Pipeline([
    ('reshape', ReshapeTransformer()),
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(hidden_layer_sizes=(100,), max_iter=500))
])

# Training Function with K-Fold Cross-Validation
def train_and_evaluate(X, y, pipelines, n_splits=5):
    """
    Trains and evaluates multiple pipelines using k-fold cross-validation.

    Parameters:
    - X: EEG data array of shape (n_samples, n_channels, n_times)
    - y: Labels array of shape (n_samples,)
    - pipelines: Dictionary of sklearn Pipelines
    - n_splits: Number of folds for cross-validation

    Returns:
    - results: Dictionary containing cross-validation scores for each pipeline
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    results = {}

    for name, pipeline in pipelines.items():
        print(f"Evaluating pipeline: {name}")
        try:
            # For EEGNet, we need to adjust the data shape inside the cross-validation
            if name == 'EEGNet_CNN':
                # Use StratifiedKFold to maintain class balance
                from sklearn.model_selection import StratifiedKFold
                skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
                scores = []
                for train_index, test_index in skf.split(X, y):
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y[train_index], y[test_index]
                    pipeline.fit(X_train, y_train)
                    score = pipeline.score(X_test, y_test)
                    scores.append(score)
                scores = np.array(scores)
            else:
                scores = cross_val_score(pipeline, X, y, cv=kf, scoring='accuracy', n_jobs=-1)
            print(f"Scores: {scores}")
            print(f"Mean accuracy: {np.mean(scores):.4f}")
            results[name] = scores
        except Exception as e:
            print(f"An error occurred in pipeline {name}: {e}")
            results[name] = None
    return results

# Example Usage
# Assuming `X` and `y` are your EEG data and labels
# Adjust the code to match your data loading and preprocessing steps

# Example call
# results = train_and_evaluate(X, y, pipelines, n_splits=5)


In [ ]:
# Import necessary libraries
import mne
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
                              VotingClassifier, StackingClassifier)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from mne.decoding import CSP, Vectorizer
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

# Import for Deep Learning models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, DepthwiseConv2D, SeparableConv2D,
                                     BatchNormalization, Activation, AveragePooling2D,
                                     Dropout, Flatten, Dense)
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# ===========================================
# Custom Transformers (Use built-in where possible)
# ===========================================

# Since some transformations are not available in built-in libraries,
# we define custom transformers only when necessary.

class ReshapeTransformer(BaseEstimator, TransformerMixin):
    """Reshapes 3D EEG data to 2D for classifier input."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        n_samples = X.shape[0]
        return X.reshape(n_samples, -1)

class PSDTransformer(BaseEstimator, TransformerMixin):
    """Extracts Power Spectral Density (PSD) features."""
    def __init__(self, sfreq=256, fmin=0.1, fmax=40):
        self.sfreq = sfreq
        self.fmin = fmin
        self.fmax = fmax
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        psd, freqs = mne.time_frequency.psd_array_multitaper(
            X, sfreq=self.sfreq, fmin=self.fmin, fmax=self.fmax, verbose=False)
        return psd.reshape(psd.shape[0], -1)

class ARTransformer(BaseEstimator, TransformerMixin):
    """Computes Autoregressive (AR) coefficients."""
    def __init__(self, order=5):
        self.order = order
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        from statsmodels.tsa.ar_model import AutoReg
        n_samples, n_channels, n_times = X.shape
        ar_features = np.zeros((n_samples, n_channels * self.order))
        for i in range(n_samples):
            for j in range(n_channels):
                model = AutoReg(X[i, j, :], lags=self.order, old_names=False)
                model_fit = model.fit()
                ar_coeffs = model_fit.params[1:]  # Exclude intercept
                ar_features[i, j * self.order:(j + 1) * self.order] = ar_coeffs
        return ar_features

# ===========================================
# Machine Learning Pipelines
# ===========================================

ml_pipelines = {}

# 1. CSP + Logistic Regression
ml_pipelines['CSP_LogReg'] = Pipeline([
    ('csp', CSP(n_components=4, reg=None, log=True, norm_trace=False)),
    ('logreg', LogisticRegression(max_iter=1000))
])

# 2. PSD + SVM
ml_pipelines['PSD_SVM'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 3. Time Domain Features + Random Forest
class TimeDomainTransformer(BaseEstimator, TransformerMixin):
    """Computes time-domain features like mean, variance, skewness."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        mean = np.mean(X, axis=2)
        var = np.var(X, axis=2)
        skewness = np.mean(((X - mean[:, :, np.newaxis]) ** 3), axis=2) / (var ** 1.5)
        features = np.concatenate((mean, var, skewness), axis=1)
        return features

ml_pipelines['TimeDomain_RF'] = Pipeline([
    ('time_features', TimeDomainTransformer()),
    ('rf', RandomForestClassifier())
])

# 4. Hilbert Transform + k-NN
class HilbertTransformer(BaseEstimator, TransformerMixin):
    """Applies the Hilbert transform to EEG data."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        analytic_signal = mne.filter.hilbert(X, picks=None, envelope=False, verbose=False)
        amplitude_envelope = np.abs(analytic_signal)
        return amplitude_envelope

ml_pipelines['Hilbert_KNN'] = Pipeline([
    ('hilbert', HilbertTransformer()),
    ('reshape', ReshapeTransformer()),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 5. Wavelet Transform + PSD + Naive Bayes
class WaveletTransformer(BaseEstimator, TransformerMixin):
    """Applies Wavelet Transform."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        import pywt
        n_samples, n_channels, n_times = X.shape
        wavelet_features = []
        for i in range(n_samples):
            sample_features = []
            for j in range(n_channels):
                coeffs = pywt.wavedec(X[i, j, :], 'db4', level=3)
                coeffs_flat = np.concatenate([c.flatten() for c in coeffs])
                sample_features.append(coeffs_flat)
            wavelet_features.append(np.concatenate(sample_features))
        return np.array(wavelet_features)

ml_pipelines['Wavelet_PSD_NB'] = Pipeline([
    ('wavelet', WaveletTransformer()),
    ('psd', PSDTransformer()),
    ('nb', GaussianNB())
])

# 6. CAR + CSP + Decision Tree
class CARTransformer(BaseEstimator, TransformerMixin):
    """Applies Common Average Referencing (CAR)."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        car = X - np.mean(X, axis=1, keepdims=True)
        return car

ml_pipelines['CAR_CSP_DT'] = Pipeline([
    ('car', CARTransformer()),
    ('csp', CSP(n_components=4, reg=None, log=True, norm_trace=False)),
    ('dt', DecisionTreeClassifier())
])

# 7. ICA + Time Domain Features + SVM
class ICATransformer(BaseEstimator, TransformerMixin):
    """Applies Independent Component Analysis (ICA)."""
    def __init__(self, n_components=15, random_state=42):
        self.n_components = n_components
        self.random_state = random_state
        self.ica = None
    def fit(self, X, y=None):
        n_samples, n_channels, n_times = X.shape
        X_concat = X.reshape(n_samples * n_channels, n_times)
        self.ica = mne.preprocessing.ICA(n_components=self.n_components,
                                         random_state=self.random_state,
                                         max_iter='auto', verbose=False)
        info = mne.create_info(ch_names=['eeg'] * X_concat.shape[0],
                               sfreq=256, ch_types='eeg')
        raw = mne.io.RawArray(X_concat, info, verbose=False)
        self.ica.fit(raw)
        return self
    def transform(self, X):
        n_samples, n_channels, n_times = X.shape
        X_transformed = []
        for i in range(n_samples):
            data = X[i]
            info = mne.create_info(ch_names=['eeg'] * n_channels,
                                   sfreq=256, ch_types='eeg')
            raw = mne.io.RawArray(data, info, verbose=False)
            raw_ica = self.ica.apply(raw.copy(), exclude=[], verbose=False)
            X_transformed.append(raw_ica.get_data())
        return np.array(X_transformed)

ml_pipelines['ICA_TimeDomain_SVM'] = Pipeline([
    ('ica', ICATransformer(n_components=15)),
    ('time_features', TimeDomainTransformer()),
    ('svm', SVC())
])

# 8. CSP + LDA
ml_pipelines['CSP_LDA'] = Pipeline([
    ('csp', CSP(n_components=4, reg=None, log=True, norm_trace=False)),
    ('lda', LDA())
])

# 9. PSD + Gradient Boosting
ml_pipelines['PSD_GB'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('gb', GradientBoostingClassifier())
])

# 10. CAR + Riemannian Geometry Features + Logistic Regression
ml_pipelines['CAR_Riemann_LogReg'] = Pipeline([
    ('car', CARTransformer()),
    ('cov', Covariances()),
    ('ts', TangentSpace()),
    ('logreg', LogisticRegression(max_iter=1000))
])

# 11. STFT + SVM
class STFTTransformer(BaseEstimator, TransformerMixin):
    """Computes Short-Time Fourier Transform (STFT)."""
    def __init__(self, n_fft=256):
        self.n_fft = n_fft
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        from scipy.signal import stft
        n_samples, n_channels, n_times = X.shape
        stft_features = []
        for i in range(n_samples):
            sample_features = []
            for j in range(n_channels):
                _, _, Zxx = stft(X[i, j, :], nperseg=self.n_fft)
                sample_features.append(np.abs(Zxx).flatten())
            stft_features.append(np.concatenate(sample_features))
        return np.array(stft_features)

ml_pipelines['STFT_SVM'] = Pipeline([
    ('stft', STFTTransformer(n_fft=256)),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 12. Morlet Wavelet Transform + k-NN
class MorletWaveletTransformer(BaseEstimator, TransformerMixin):
    """Applies time-frequency analysis using Morlet wavelets."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        freqs = np.linspace(8, 30, num=22)
        n_samples, n_channels, n_times = X.shape
        power_features = []
        for i in range(n_samples):
            sample_features = []
            for j in range(n_channels):
                power = mne.time_frequency.tfr_array_morlet(
                    X[i:i+1, j:j+1, :], sfreq=256, freqs=freqs,
                    n_cycles=7, output='power', verbose=False)
                sample_features.append(power.flatten())
            power_features.append(np.concatenate(sample_features))
        return np.array(power_features)

ml_pipelines['Morlet_KNN'] = Pipeline([
    ('morlet', MorletWaveletTransformer()),
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 13. AR Coefficients + Logistic Regression
ml_pipelines['AR_LogReg'] = Pipeline([
    ('ar', ARTransformer(order=5)),
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000))
])

# 14. Mean Amplitude Features + Naive Bayes
class MeanAmplitudeTransformer(BaseEstimator, TransformerMixin):
    """Computes the mean amplitude of the signal."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        mean_amplitude = np.mean(np.abs(X), axis=2)
        return mean_amplitude

ml_pipelines['MeanAmp_NB'] = Pipeline([
    ('mean_amp', MeanAmplitudeTransformer()),
    ('nb', GaussianNB())
])

# 15. ICA + PSD + SVM
ml_pipelines['ICA_PSD_SVM'] = Pipeline([
    ('ica', ICATransformer(n_components=15)),
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 16. FFT + Random Forest
class FFTTransformer(BaseEstimator, TransformerMixin):
    """Computes Fast Fourier Transform (FFT)."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        fft_coeffs = np.fft.rfft(X, axis=2)
        fft_features = np.abs(fft_coeffs)
        return fft_features.reshape(X.shape[0], -1)

ml_pipelines['FFT_RF'] = Pipeline([
    ('fft', FFTTransformer()),
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier())
])

# 17. Time-Frequency Features (STFT) + LDA
ml_pipelines['TimeFreq_LDA'] = Pipeline([
    ('stft', STFTTransformer(n_fft=256)),
    ('scaler', StandardScaler()),
    ('lda', LDA())
])

# 18. PSD Features + XGBoost Classifier
ml_pipelines['PSD_XGB'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
])

# 19. Ensemble Methods (Stacking Classifier)
ml_pipelines['Stacking'] = Pipeline([
    ('psd', PSDTransformer()),
    ('scaler', StandardScaler()),
    ('stacking', StackingClassifier(
        estimators=[
            ('svm', SVC(probability=True)),
            ('rf', RandomForestClassifier()),
            ('knn', KNeighborsClassifier())
        ],
        final_estimator=LogisticRegression(max_iter=1000),
        cv=5
    ))
])

# 20. Sparse Representation Classification (SRC)
class SRCClassifier(BaseEstimator, ClassifierMixin):
    """Simulates a Sparse Representation Classifier using Lasso."""
    def __init__(self, alpha=0.1):
        self.alpha = alpha
        self.classes_ = None
        self.dictionary_ = None
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.dictionary_ = {}
        for cls in self.classes_:
            self.dictionary_[cls] = X[y == cls].T
        return self
    def predict(self, X):
        preds = []
        for x in X:
            residuals = []
            for cls in self.classes_:
                lasso = Lasso(alpha=self.alpha, max_iter=1000)
                lasso.fit(self.dictionary_[cls], x)
                reconstruction = lasso.predict(self.dictionary_[cls])
                residual = np.linalg.norm(x - reconstruction)
                residuals.append(residual)
            preds.append(self.classes_[np.argmin(residuals)])
        return np.array(preds)

ml_pipelines['SRC'] = Pipeline([
    ('reshape', ReshapeTransformer()),
    ('scaler', StandardScaler()),
    ('src', SRCClassifier(alpha=0.1))
])

# 21. Multilayer Perceptron (MLP) Neural Network (As ML Pipeline)
ml_pipelines['MLP_NN'] = Pipeline([
    ('reshape', ReshapeTransformer()),
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(hidden_layer_sizes=(100,), max_iter=500))
])

# ===========================================
# Neural Network Pipelines
# ===========================================

nn_pipelines = {}

# 1. EEGNet (Deep CNN)
class EEGNetTransformer(BaseEstimator, TransformerMixin):
    """Transformer that reshapes data for EEGNet."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # Reshape X to (n_samples, n_channels, n_times, 1)
        n_samples, n_channels, n_times = X.shape
        X_reshaped = X.reshape(n_samples, n_channels, n_times, 1)
        return X_reshaped

def create_eegnet_model(n_channels, n_times, n_classes):
    """Creates an EEGNet model."""
    input_shape = (n_channels, n_times, 1)
    inputs = Input(shape=input_shape)

    # Block 1
    x = Conv2D(16, (1, 64), padding='same',
               use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = DepthwiseConv2D((n_channels, 1), use_bias=False,
                        depth_multiplier=2,
                        depthwise_constraint=max_norm(1.))(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)
    x = AveragePooling2D((1, 4))(x)
    x = Dropout(0.25)(x)

    # Block 2
    x = SeparableConv2D(32, (1, 16), use_bias=False, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('elu')(x)
    x = AveragePooling2D((1, 8))(x)
    x = Dropout(0.25)(x)

    x = Flatten()(x)

    x = Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Placeholder function to create the model with the correct input dimensions
def create_eegnet_model_wrapper():
    n_channels = X.shape[1]
    n_times = X.shape[2]
    n_classes = len(np.unique(y))
    return create_eegnet_model(n_channels, n_times, n_classes)

nn_pipelines['EEGNet_CNN'] = Pipeline([
    ('reshape', EEGNetTransformer()),
    ('eegnet', KerasClassifier(build_fn=create_eegnet_model_wrapper, epochs=50, batch_size=16, verbose=0))
])

# ===========================================
# Training Function with K-Fold Cross-Validation
# ===========================================

def train_and_evaluate(X, y, pipelines, n_splits=5):
    """
    Trains and evaluates multiple pipelines using k-fold cross-validation.

    Parameters:
    - X: EEG data array of shape (n_samples, n_channels, n_times)
    - y: Labels array of shape (n_samples,)
    - pipelines: Dictionary of sklearn Pipelines
    - n_splits: Number of folds for cross-validation

    Returns:
    - results: Dictionary containing cross-validation scores for each pipeline
    """
    results = {}

    for name, pipeline in pipelines.items():
        print(f"Evaluating pipeline: {name}")
        try:
            if name == 'EEGNet_CNN':
                # Use StratifiedKFold for EEGNet to maintain class balance
                skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
                scores = []
                for train_index, test_index in skf.split(X, y):
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y[train_index], y[test_index]
                    pipeline.fit(X_train, y_train)
                    score = pipeline.score(X_test, y_test)
                    scores.append(score)
                scores = np.array(scores)
            else:
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
                scores = cross_val_score(pipeline, X, y, cv=kf, scoring='accuracy', n_jobs=-1)
            print(f"Scores: {scores}")
            print(f"Mean accuracy: {np.mean(scores):.4f}")
            results[name] = scores
        except Exception as e:
            print(f"An error occurred in pipeline {name}: {e}")
            results[name] = None
    return results

# ===========================================
# Example Usage
# ===========================================

# Assuming `X` and `y` are your EEG data and labels
# Adjust the code to match your data loading and preprocessing steps

# Example call for Machine Learning Pipelines
# results_ml = train_and_evaluate(X, y, ml_pipelines, n_splits=5)

# Example call for Neural Network Pipelines
# results_nn = train_and_evaluate(X, y, nn_pipelines, n_splits=5)


# Explanation of Pipelines for Mental Imagery BCI

Below is a detailed explanation of each of the pipelines mentioned. For each pipeline, I'll describe:

- **Preprocessing Steps**: Any data preprocessing or transformations applied before feature extraction.
- **Feature Extraction**: Methods used to extract features from the EEG data.
- **Classifier**: The machine learning algorithm used for classification.
- **Notes**: Any additional information or considerations.

---

## 1. Common Spatial Patterns (CSP) + Logistic Regression (`CSP_LogReg`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Common Spatial Patterns (CSP)**:
  - **Purpose**: CSP is a feature extraction method that projects multi-channel EEG data into a low-dimensional spatial subspace that maximizes the variance for one class while minimizing it for the other.
  - **How It Works**: CSP computes spatial filters that optimize the discriminability between two classes by maximizing the variance of one class while minimizing the variance of the other.
  - **Output**: CSP transforms the EEG data into a set of features (usually the log-variance of the filtered signals), resulting in a feature matrix suitable for classification.

### Classifier

- **Logistic Regression**:
  - A linear classifier that models the probability of class membership using the logistic function.
  - **Advantages**: Simple, interpretable, and performs well with linearly separable data.

### Notes

- **CSP Assumptions**: Works best when data is bandpass filtered to relevant frequency bands (e.g., mu and beta rhythms).
- **Binary Classification**: CSP is typically used for binary classification tasks.

---

## 2. Power Spectral Density (PSD) + Support Vector Machine (SVM) (`PSD_SVM`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Power Spectral Density (PSD)**:
  - **Purpose**: PSD measures the power of the EEG signal at different frequency components.
  - **How It Works**: Using methods like Welch's method or multitaper, the EEG data is transformed into the frequency domain, and the power at each frequency bin is calculated.
  - **Output**: A feature vector representing the power distribution across frequencies for each channel.

### Classifier

- **Support Vector Machine (SVM)**:
  - A supervised learning algorithm that finds the optimal hyperplane to separate classes.
  - **Kernel Trick**: Can use different kernel functions (linear, polynomial, RBF) to handle non-linearly separable data.

### Notes

- **Feature Dimensionality**: PSD features can be high-dimensional; dimensionality reduction or feature selection may be beneficial.
- **SVM Advantages**: Effective in high-dimensional spaces and with clear margin separation.

---

## 3. Time Domain Features + Random Forest (`TimeDomain_RF`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Time-Domain Features**:
  - **Mean**: Average value of the EEG signal over time.
  - **Variance**: Measure of signal variability.
  - **Skewness**: Measure of asymmetry in the signal distribution.
  - **How It Works**: These statistical features are computed across the time dimension for each channel.

### Classifier

- **Random Forest**:
  - An ensemble method using multiple decision trees.
  - **How It Works**: Each tree is trained on a bootstrap sample with random feature selection; the final prediction is made by aggregating the outputs of all trees (e.g., majority vote).

### Notes

- **Non-linear Relationships**: Random forests can capture non-linear relationships between features and target classes.
- **Feature Importance**: Random forests provide measures of feature importance, useful for feature selection.

---

## 4. Hilbert Transform + k-Nearest Neighbors (k-NN) (`Hilbert_KNN`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Hilbert Transform**:
  - **Purpose**: To obtain the analytic signal from a real-valued signal, allowing extraction of instantaneous amplitude and phase.
  - **How It Works**: The Hilbert transform is applied to the EEG signal to compute the amplitude envelope or phase information.
  - **Output**: Amplitude envelope of the EEG signal for each channel.

### Classifier

- **k-Nearest Neighbors (k-NN)**:
  - A non-parametric classifier that assigns a class based on the majority class among the k nearest neighbors in the feature space.
  - **Distance Metrics**: Commonly uses Euclidean distance, but other metrics can be applied.

### Notes

- **Parameter Tuning**: The choice of `k` and the distance metric can significantly impact performance.
- **Computational Cost**: k-NN can be computationally expensive for large datasets.

---

## 5. Wavelet Transform + PSD + Naive Bayes (`Wavelet_PSD_NB`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Wavelet Transform**:
  - **Purpose**: Decomposes the EEG signal into time-frequency components with good time and frequency resolution.
  - **How It Works**: Applies wavelet decomposition (e.g., using Daubechies wavelets) to capture signal characteristics at various scales.
  - **Output**: Wavelet coefficients representing the signal at different scales and positions.
- **Power Spectral Density (PSD)**:
  - Calculated on the wavelet coefficients to obtain power distribution across scales.

### Classifier

- **Naive Bayes**:
  - A probabilistic classifier based on Bayes' theorem, assuming feature independence.
  - **Advantages**: Simple, fast, and works well with high-dimensional data.

### Notes

- **Feature Independence Assumption**: Naive Bayes assumes features are independent, which may not hold for EEG data.
- **Wavelet Selection**: The choice of wavelet function and decomposition level can affect feature quality.

---

## 6. Common Average Referencing (CAR) + CSP + Decision Tree (`CAR_CSP_DT`)

### Preprocessing Steps

- **Common Average Referencing (CAR)**:
  - **Purpose**: To reduce common noise across all channels.
  - **How It Works**: Subtracts the average signal across all channels from each channel's signal.
  - **Output**: Referenced EEG data with reduced artifacts.

### Feature Extraction

- **Common Spatial Patterns (CSP)**:
  - As described in Pipeline 1.

### Classifier

- **Decision Tree**:
  - A tree-based classifier that splits data based on feature thresholds to maximize class separation.
  - **Advantages**: Simple to interpret, handles both numerical and categorical data.

### Notes

- **Overfitting Risk**: Decision trees can overfit; pruning or setting maximum depth can mitigate this.
- **CAR Benefits**: CAR can enhance signal-to-noise ratio, benefiting CSP feature extraction.

---

## 7. Independent Component Analysis (ICA) + Time Domain Features + SVM (`ICA_TimeDomain_SVM`)

### Preprocessing Steps

- **Independent Component Analysis (ICA)**:
  - **Purpose**: To separate mixed signals into statistically independent components.
  - **How It Works**: Decomposes EEG signals into independent sources, which can help isolate artifacts (e.g., eye blinks, muscle movements).
  - **Output**: Cleaned EEG data or components representing neural activity.

### Feature Extraction

- **Time-Domain Features**:
  - As described in Pipeline 3.

### Classifier

- **Support Vector Machine (SVM)**:
  - As described in Pipeline 2.

### Notes

- **Artifact Removal**: ICA can effectively remove artifacts when components corresponding to noise are identified and excluded.
- **Complexity**: ICA requires careful application to avoid removing neural signals of interest.

---

## 8. CSP + Linear Discriminant Analysis (LDA) (`CSP_LDA`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Common Spatial Patterns (CSP)**:
  - As described in Pipeline 1.

### Classifier

- **Linear Discriminant Analysis (LDA)**:
  - A linear classifier that projects data onto a line to maximize class separation.
  - **How It Works**: Finds a linear combination of features that best separates the classes.

### Notes

- **Popular Combination**: CSP + LDA is a classic approach in motor imagery BCI applications due to its simplicity and effectiveness.
- **Assumptions**: LDA assumes normally distributed features with equal covariance matrices for each class.

---

## 9. PSD + Gradient Boosting Classifier (`PSD_GB`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Power Spectral Density (PSD)**:
  - As described in Pipeline 2.

### Classifier

- **Gradient Boosting Classifier**:
  - An ensemble method that builds sequential weak learners (typically decision trees), where each new learner focuses on correcting the errors of previous ones.
  - **Advantages**: High performance, can model complex relationships.

### Notes

- **Hyperparameter Tuning**: Requires careful tuning of parameters like learning rate, number of estimators, and tree depth.
- **Overfitting Risk**: Can overfit if not properly regularized.

---

## 10. CAR + Riemannian Geometry Features + Logistic Regression (`CAR_Riemann_LogReg`)

### Preprocessing Steps

- **Common Average Referencing (CAR)**:
  - As described in Pipeline 6.

### Feature Extraction

- **Riemannian Geometry Features**:
  - **Covariance Matrices**: Compute the covariance matrix of EEG signals for each trial.
  - **Tangent Space Mapping**: Maps covariance matrices to the tangent space of the Riemannian manifold to obtain feature vectors.
  - **Purpose**: Captures the spatial covariance structure of EEG signals, which is informative for classification.

### Classifier

- **Logistic Regression**:
  - As described in Pipeline 1.

### Notes

- **Riemannian Methods**: Effective for BCI applications due to robustness to noise and ability to capture complex signal structures.
- **Computational Cost**: Calculating covariance matrices and tangent space mapping can be computationally intensive.

---

## 11. Short-Time Fourier Transform (STFT) + SVM (`STFT_SVM`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Short-Time Fourier Transform (STFT)**:
  - **Purpose**: Analyzes the signal's frequency content over time by applying the Fourier Transform to short overlapping time windows.
  - **How It Works**: Divides the EEG signal into segments, applies windowing, and computes the Fourier Transform for each segment.
  - **Output**: Time-frequency representation of the signal.

### Classifier

- **Support Vector Machine (SVM)**:
  - As described in Pipeline 2.

### Notes

- **Feature Dimensionality**: STFT features can be very high-dimensional; dimensionality reduction may be necessary.
- **Time-Frequency Analysis**: Useful for capturing non-stationary properties of EEG signals.

---

## 12. Laplacian Spatial Filtering + Wavelet Packet Decomposition (WPD) + Random Forest (`Laplacian_WPD_RF`)

### Preprocessing Steps

- **Laplacian Spatial Filtering**:
  - **Purpose**: Enhances spatial resolution by emphasizing local activity and reducing volume conduction effects.
  - **How It Works**: Computes the difference between a channel and the average of its neighboring channels.
  - **Output**: Spatially filtered EEG data.

### Feature Extraction

- **Wavelet Packet Decomposition (WPD)**:
  - **Purpose**: Decomposes the signal into a set of frequency subbands with both time and frequency localization.
  - **How It Works**: Applies wavelet packet analysis to the EEG data, providing a more detailed frequency analysis than standard wavelet transforms.
  - **Output**: Coefficients representing the signal's energy at various scales and positions.

### Classifier

- **Random Forest**:
  - As described in Pipeline 3.

### Notes

- **Information Preservation**: WPD can capture subtle signal features that may be relevant for classification.
- **Computational Complexity**: WPD can be computationally intensive due to the extensive decomposition.

---

## 13. Morlet Wavelets + k-NN (`Morlet_KNN`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Time-Frequency Analysis using Morlet Wavelets**:
  - **Purpose**: Captures time-frequency characteristics of EEG signals with high resolution.
  - **How It Works**: Applies Morlet wavelet transform, which provides a complex exponential modulated by a Gaussian, ideal for EEG analysis.
  - **Output**: Power spectra across frequencies and time.

### Classifier

- **k-Nearest Neighbors (k-NN)**:
  - As described in Pipeline 4.

### Notes

- **Frequency Selection**: Frequencies of interest (e.g., 8-30 Hz) are analyzed.
- **Data Dimensionality**: The resulting features may be high-dimensional, affecting computational cost.

---

## 14. Autoregressive (AR) Coefficients + Logistic Regression (`AR_LogReg`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Autoregressive (AR) Coefficients**:
  - **Purpose**: Models the EEG signal as a linear function of its previous values.
  - **How It Works**: Fits an AR model to the EEG time series, extracting the coefficients as features.
  - **Output**: AR coefficients for each channel.

### Classifier

- **Logistic Regression**:
  - As described in Pipeline 1.

### Notes

- **Model Order**: The choice of AR model order (number of lags) affects feature quality.
- **Stationarity Assumption**: AR models assume signal stationarity within the analysis window.

---

## 15. Mean Amplitude Features + Naive Bayes (`MeanAmp_NB`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Mean Amplitude**:
  - **Purpose**: Simple feature representing the average absolute value of the EEG signal.
  - **How It Works**: Computes the mean of the absolute values of the EEG signal over time for each channel.
  - **Output**: Feature vector of mean amplitudes.

### Classifier

- **Naive Bayes**:
  - As described in Pipeline 5.

### Notes

- **Simplicity**: This approach uses straightforward features and a simple classifier.
- **Performance**: May not capture complex signal characteristics, potentially limiting classification accuracy.

---

## 16. ICA + PSD + SVM (`ICA_PSD_SVM`)

### Preprocessing Steps

- **Independent Component Analysis (ICA)**:
  - As described in Pipeline 7.

### Feature Extraction

- **Power Spectral Density (PSD)**:
  - Calculated on the ICA components to analyze the frequency content of the independent sources.
  - **Output**: PSD features derived from the independent components.

### Classifier

- **Support Vector Machine (SVM)**:
  - As described in Pipeline 2.

### Notes

- **Artifact Removal**: ICA can help isolate neural signals from artifacts before feature extraction.
- **Component Selection**: Choosing relevant components is crucial; including artifact components can degrade performance.

---

## 17. Fast Fourier Transform (FFT) + Random Forest (`FFT_RF`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Fast Fourier Transform (FFT)**:
  - **Purpose**: Transforms the time-domain EEG signal into the frequency domain.
  - **How It Works**: Computes the discrete Fourier Transform efficiently to obtain frequency coefficients.
  - **Output**: Magnitude of the FFT coefficients for each channel.

### Classifier

- **Random Forest**:
  - As described in Pipeline 3.

### Notes

- **Frequency Resolution**: The length of the FFT determines the frequency resolution.
- **Feature Selection**: Not all frequency components may be informative; selecting relevant frequencies can improve performance.

---

## 18. Time-Frequency Features + Linear Discriminant Analysis (LDA) (`TimeFreq_LDA`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Time-Frequency Analysis (e.g., STFT or Wavelet Transform)**:
  - **Purpose**: Captures how the frequency content of the EEG signal changes over time.
  - **How It Works**: Applies time-frequency transformations to extract features representing both temporal and spectral information.
  - **Output**: Time-frequency representation flattened into feature vectors.

### Classifier

- **Linear Discriminant Analysis (LDA)**:
  - As described in Pipeline 8.

### Notes

- **Dimensionality Reduction**: Due to high dimensionality, techniques like PCA may be applied before classification.
- **Temporal Dynamics**: Time-frequency features can capture transient patterns associated with mental imagery.

---

## 19. EEGNet (Deep Convolutional Neural Network) (`EEGNet_CNN`)

### Preprocessing Steps

- **Assumption**: No external preprocessing; EEGNet handles preprocessing internally.

### Feature Extraction and Classification

- **EEGNet Architecture**:
  - **Purpose**: A compact CNN architecture tailored for EEG-based BCIs.
  - **How It Works**: Consists of convolutional layers that learn spatial and temporal filters, depthwise and separable convolutions to reduce parameters, and fully connected layers for classification.
  - **Input Shape**: Expects input data in the shape `(n_samples, n_channels, n_times, 1)`.

### Notes

- **End-to-End Learning**: Learns feature extraction and classification jointly.
- **Computational Resources**: Requires more computational power and training time compared to traditional methods.
- **Data Requirements**: Deep learning models generally require large amounts of data to avoid overfitting.

---

## 20. PSD Features + XGBoost Classifier (`PSD_XGB`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Power Spectral Density (PSD)**:
  - As described in Pipeline 2.

### Classifier

- **XGBoost Classifier**:
  - An optimized gradient boosting algorithm.
  - **Advantages**: High performance, regularization to prevent overfitting, handles missing values.

### Notes

- **Hyperparameter Tuning**: XGBoost has many parameters that can be tuned to optimize performance.
- **Feature Importance**: Provides measures of feature importance, aiding in feature selection.

---

## 21. Ensemble Methods (Stacking Classifier) (`Stacking`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Power Spectral Density (PSD)**:
  - As described in Pipeline 2.

### Classifier

- **Stacking Classifier**:
  - **Base Learners**: Combines predictions from multiple classifiers (e.g., SVM, Random Forest, k-NN).
  - **Meta-Learner**: Uses a higher-level classifier (e.g., Logistic Regression) to make the final prediction based on base learners' outputs.
  - **Advantages**: Can capture diverse patterns by leveraging strengths of different classifiers.

### Notes

- **Complexity**: Requires careful management to avoid overfitting due to increased model complexity.
- **Cross-Validation**: Internal cross-validation is often used to prevent information leakage between training and validation data.

---

## 22. Sparse Representation Classification (SRC) (`SRC`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Raw Data or CSP Features**:
  - **Purpose**: SRC can work with raw data or features extracted using methods like CSP.
  - **How It Works**: Each class's training samples form a dictionary; test samples are represented as sparse linear combinations of dictionary atoms.

### Classifier

- **Sparse Representation Classifier (SRC)**:
  - **How It Works**: Solves an optimization problem to find the sparsest representation of a test sample in terms of the training dictionary.
  - **Prediction**: Classifies based on which class's dictionary best represents the test sample.

### Notes

- **Computational Complexity**: SRC can be computationally intensive due to the optimization involved.
- **Implementation**: In practice, approximations or simplifications (e.g., using Lasso regression) may be used.

---

## 23. Multilayer Perceptron (MLP) Neural Network (`MLP_NN`)

### Preprocessing Steps

- **Assumption**: Bandpass filtering is performed separately before this pipeline.

### Feature Extraction

- **Flattened EEG Data or Features**:
  - The EEG data is reshaped into a 2D array (samples x features), either directly or after feature extraction.

### Classifier

- **MLP Neural Network**:
  - **Architecture**: Consists of input, hidden, and output layers with non-linear activation functions.
  - **Learning**: Uses backpropagation to adjust weights and biases.

### Notes

- **Flexibility**: MLPs can model complex non-linear relationships.
- **Overfitting Risk**: Prone to overfitting if the network is too large or data is insufficient.
- **Hyperparameters**: Number of layers, neurons, activation functions, and learning rate need to be tuned.

---

## General Considerations Across Pipelines

- **Data Preprocessing**:
  - **Bandpass Filtering**: Essential to focus on frequency bands relevant to mental imagery (e.g., alpha, beta rhythms).
  - **Artifact Removal**: Techniques like ICA and CAR help reduce noise and artifacts, improving feature quality.

- **Feature Extraction**:
  - **Importance**: The choice of feature extraction method significantly impacts classification performance.
  - **Dimensionality**: High-dimensional features may require dimensionality reduction or regularization to prevent overfitting.

- **Classifier Selection**:
  - **Linear vs. Non-linear**: Linear classifiers like LDA and Logistic Regression are simple and interpretable but may not capture complex patterns.
  - **Ensemble Methods**: Can improve performance by combining multiple models but may increase computational cost.
  - **Deep Learning Models**: Offer powerful feature learning capabilities but require more data and computational resources.

- **Evaluation**:
  - **Cross-Validation**: Essential for assessing model performance and generalization ability.
  - **Hyperparameter Tuning**: Optimization of model parameters is crucial for achieving the best performance.

---

## Conclusion

Each pipeline combines specific preprocessing techniques, feature extraction methods, and classifiers tailored to capture the characteristics of EEG signals associated with mental imagery tasks. The choice of pipeline depends on factors such as the nature of the EEG data, computational resources, and the specific requirements of the BCI application. By experimenting with different pipelines, researchers can identify the most effective approaches for their particular datasets and objectives.

---

If you have any questions about any of these pipelines or need further clarification on specific components, feel free to ask!


In [ ]:
import mne
import numpy as np
import scipy.signal as signal
from numpy import ndarray
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class LogVarianceTransformer(BaseEstimator, TransformerMixin):
    """Computes the log-variance of each channel over time."""
    def fit(self, X, y=None):
        """Fit transformer. `y` is ignored."""
        return self
    def transform(self, X):
        """Compute log-variance of the input data.

        Parameters:
        - X: array-like, shape (n_samples, n_channels, n_times)
        
        Returns:
        - log_var: array, shape (n_samples, n_channels)
        """
        assert X.ndim == 3, "Input data X must be a 3D array."
        log_var = np.log(np.var(X, axis=-1))
        return log_var


class LogVarianceTransformer(BaseEstimator, TransformerMixin):
    """Computes the log-variance of each channel."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # X is of shape (n_samples, n_channels, n_times)
        log_var = np.log(np.var(X, axis=2))
        return log_var
    
class LogVariance(BaseEstimator, TransformerMixin):
    """LogVariance transformer."""

    def fit(self, X, y):
        """fit."""
        return self

    def transform(self, X):
        """transform."""
        assert X.ndim == 3
        return np.log(np.var(X, -1))


class FM(BaseEstimator, TransformerMixin):
    """Transformer to scale sampling frequency."""

    def __init__(self, freq=128):
        """Init function for FM transformer.

        Instantaneous frequencies require a sampling frequency to be
        properly scaled, which is helpful for some algorithms.

        This assumes 128 if not told otherwise.

        Parameters
        ----------
        freq: int
            Sampling frequency of the signal. This is used to scale
            the instantaneous frequency.
        """
        self.freq = freq

    def fit(self, X, y):
        """Only for scikit-learn compatibility."""
        return self

    def transform(self, X):
        """transform."""
        xphase = np.unwrap(np.angle(signal.hilbert(X, axis=-1)))
        return np.median(self.freq * np.diff(xphase, axis=-1) / (2 * np.pi), axis=-1)
